# 管理分区

本指南指导您如何创建和管理集合中的分区。

## 概述

Milvus中的分区表示集合的细分。此功能允许将集合的物理存储划分为多个部分，通过将焦点缩小到较小的数据子集而不是整个集合来提高查询性能。

在创建集合时，至少会自动创建一个名为_default的默认分区。在一个集合中最多可以创建4，096个分区。

**注意到**

Milvus引入了一个名为Partition Key的功能，利用底层分区来存储基于特定字段的散列值的实体。此功能有助于实现多租户，提高搜索性能。有关详细信息，请参阅[使用分区键](https://milvus.io/docs/use-partition-key.md)。

如果集合中的分区键功能是打开的，Milvus会负责管理所有分区，从而减轻您的责任。

此页面上的代码片段使用新[的MilvusClient](https://milvus.io/api-reference/pymilvus/v2.4.x/About.md)（Python）与Milvus交互。其他语言的新MilvusClient SDK将在未来的更新中发布。

## 筹备工作

下面的代码片段重新使用了现有的代码，以建立到Milvus的连接，并在快速设置模式下创建一个集合，指示该集合在创建时加载。


In [1]:
from pymilvus import MilvusClient, DataType

# 1. Set up a Milvus client
client = MilvusClient(
    uri="http://localhost:19530"
)

# 2. Create a collection
client.create_collection(
    collection_name="quick_setup",
    dimension=5,
)

**注意到**

在上面的代码片段中，集合的索引已经与集合一起沿着创建，这表明集合是在创建时加载的。

## 列出分区

一旦集合准备就绪，您就可以列出它的分区。


In [2]:
# 3. List partitions
res = client.list_partitions(collection_name="quick_setup")
print(res)

['_default']


上面代码片段的输出包括指定集合中分区的名称。

**注意到**

如果您将一个字段设置为集合中的分区键，Milvus将与该集合一起创建至少64个分区沿着。列出分区时，结果可能与上述代码片段的输出不同。

有关详细信息，请参阅[使用分区密钥](https://milvus.io/docs/use-partition-key.md)。

## 创建分区


您可以向集合中添加更多分区。一个集合最多可以有64个分区。

In [3]:
# 4. Create more partitions
client.create_partition(
    collection_name="quick_setup",
    partition_name="partitionA"
)

client.create_partition(
    collection_name="quick_setup",
    partition_name="partitionB"
)

res = client.list_partitions(collection_name="quick_setup")
print(res)

['_default', 'partitionA', 'partitionB']


上面的代码片段在集合中创建了一个分区，并列出了集合的分区。

**注意到**

如果您将字段设置为集合中的分区键，Milvus将负责管理集合中的分区。因此，在尝试创建分区时可能会遇到提示错误。

有关详细信息，请参阅[使用分区密钥](https://milvus.io/docs/use-partition-key.md)。


## 检查特定分区

您还可以检查特定分区的存在。

In [4]:
# 5. Check whether a partition exists
res = client.has_partition(
    collection_name="quick_setup",
    partition_name="partitionA"
)
print(res)

True


In [5]:
res = client.has_partition(
    collection_name="quick_setup",
    partition_name="partitionC"
)
print(res)

False


上面的代码片段检查集合是否有一个名为`partitionA`和`partitionC`的分区。


## 加载和释放分区



您可以加载和释放特定的分区，以使它们可用于或不可用于搜索和查询。

### 获取加载状态



要检查集合及其分区的加载状态，请执行以下操作：

In [6]:
# Release the collection
client.release_collection(collection_name="quick_setup")

# Check the load status
res = client.get_load_state(collection_name="quick_setup")
print(res)

{'state': <LoadState: NotLoad>}


In [7]:
res = client.get_load_state(
    collection_name="quick_setup", 
    partition_name="partitionA"
)
print(res)

{'state': <LoadState: NotLoad>}


In [8]:
res = client.get_load_state(
    collection_name="quick_setup", 
    partition_name="partitionB"
)

print(res)

{'state': <LoadState: NotLoad>}


可能的加载状态可能是以下任一状态

- **Loaded 加载**

  如果集合中至少有一个分区已被加载，则该集合被标记为`Loaded`。

- **NotLoad NotLoad**

  如果一个集合的分区都没有被加载，则该集合被标记为`NotLoad`。

- **Loading 加载中**

### 加载分区

要加载集合的所有分区，您只需调用`load_collection()`。要加载集合的特定分区，请执行以下操作：


In [9]:
client.load_partitions(
    collection_name="quick_setup",
    partition_names=["partitionA"]
)

res = client.get_load_state(collection_name="quick_setup")
print(res)

{'state': <LoadState: Loaded>}


要一次加载多个分区，请执行以下操作：

In [13]:
client.load_partitions(
    collection_name="quick_setup",
    partition_names=["partitionA", "partitionB"]
)

res = client.get_load_state(
    collection_name="quick_setup",
    partition_name="partitionA"
)
res

{'state': <LoadState: Loaded>}

In [14]:
res = client.get_load_state(
    collection_name="quick_setup",
    partition_name="partitionB"
)
res

{'state': <LoadState: Loaded>}

### 释放分区

要释放集合的所有分区，您只需调用`release_collection`。要释放集合的特定分区，请执行以下操作：


In [15]:
# 7. Release a partition
client.release_partitions(
    collection_name="quick_setup",
    partition_names=["partitionA"]
)

res = client.get_load_state(
    collection_name="quick_setup", 
    partition_name="partitionA"
)

print(res)


{'state': <LoadState: NotLoad>}


要一次释放多个分区，请执行以下操作：

In [17]:
client.release_partitions(
    collection_name="quick_setup",
    partition_names=["_default", "partitionA", "partitionB"]
)

res = client.get_load_state(
    collection_name="quick_setup",
)

res

{'state': <LoadState: NotLoad>}

## 删除分区


释放分区后，如果不再需要它，可以将其删除。

In [18]:
# 8. Drop a partition
client.drop_partition(
    collection_name="quick_setup",
    partition_name="partitionB"
)

res = client.list_partitions(collection_name="quick_setup")
print(res)


['_default', 'partitionA']


**注意到**

在删除分区之前，您需要将其从内存中释放。

## FAQ

- **一个分区可以存储多少数据？**

  建议在分区中存储少于1B的数据。

- **可以创建的分区的最大数量是多少？**

  默认情况下，Milvus最多允许创建4，096个分区。您可以通过配置`rootCoord.maxPartitionNum`来调整最大分区数。有关详细信息，请参阅[系统手册](https://milvus.io/docs/configure_rootcoord.md#rootCoordmaxPartitionNum)。

- **如何区分分区和分区键？**

  分区是物理存储单元，而分区键是逻辑概念，它根据指定的列自动将数据分配给特定的分区。

  例如，在Milvus中，如果您有一个集合，其分区键定义为`color`字段，则系统会根据每个实体的`color`字段的散列值自动将数据分配给分区。这个自动化的过程减轻了用户在插入或搜索数据时手动指定分区的责任。

  另一方面，在手动创建分区时，您需要根据分区键的标准将数据分配给每个分区。如果您有一个包含`color`字段的集合，则可以手动将实体的`color`值从`red`分配到`partition A`，将实体的`color`值从`blue`分配到`partition B`。这种手动管理需要更多的努力。

  总之，分区和分区键都被用来优化数据计算和提高查询效率。必须认识到，启用分区键意味着放弃对分区数据插入和加载的手动管理的控制，因为这些过程是完全自动化的，由Milvus处理。

------